Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [11]:
# Copy from assignment 2

graph = tf.Graph()
batch_size = 128
hidden_size = 1024

with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      weights_h = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_size]))
      biases_h = tf.Variable(tf.zeros([hidden_size]))
      weights_y = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
      biases_y = tf.Variable(tf.truncated_normal([num_labels]))

      # Training computation.
      def single_hidden_layer_logit(input_dataset):
        hidden = tf.nn.relu(tf.matmul(input_dataset, weights_h) + biases_h)
        logits = tf.matmul(hidden, weights_y) + biases_y
        return logits

      logits = single_hidden_layer_logit(tf_train_dataset)
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
      loss_with_reg = loss + reg_term * (tf.nn.l2_loss(weights_h) + tf.nn.l2_loss(weights_y))

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss_with_reg)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(single_hidden_layer_logit(tf_valid_dataset))
      test_prediction = tf.nn.softmax(single_hidden_layer_logit(tf_test_dataset))

In [ ]:
num_steps = 3001

def run_graph():
    with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print "Initialized"
      for step in xrange(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print "Minibatch loss at step", step, ":", l
          print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
          print "Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels)
      print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

for reg_term in [1., 0.3, 0.1, 0.03, 0.01]:
    setup_graph(reg_term)
    print ("Training with reg_term %.2f" % reg_term)
    run_graph()

Training with reg_term 1.00
Initialized
Minibatch loss at step 0 : 358.72
Minibatch accuracy: 10.9%
Validation accuracy: 23.9%
Minibatch loss at step 500 : 2.30496
Minibatch accuracy: 7.8%
Validation accuracy: 8.8%
Minibatch loss at step 1000 : 2.30289
Minibatch accuracy: 9.4%
Validation accuracy: 9.8%
Minibatch loss at step 1500 : 2.3049
Minibatch accuracy: 9.4%
Validation accuracy: 10.4%
Minibatch loss at step 2000 : 2.30453
Minibatch accuracy: 10.9%
Validation accuracy: 10.3%
Minibatch loss at step 2500 : 2.30855
Minibatch accuracy: 8.6%
Validation accuracy: 10.3%
Minibatch loss at step 3000 : 2.30002
Minibatch accuracy: 10.9%
Validation accuracy: 9.6%
Test accuracy: 10.0%
Training with reg_term 0.30
Initialized
Minibatch loss at step 0 : 393.958
Minibatch accuracy: 5.5%
Validation accuracy: 21.0%
Minibatch loss at step 500 : 2.30511
Minibatch accuracy: 7.8%
Validation accuracy: 10.1%
Minibatch loss at step 1000 : 2.3029
Minibatch accuracy: 9.4%
Validation accuracy: 9.8%
Minibatch l

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [ ]:
num_steps = 3001

def run_graph():
    with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      train_dataset_small = train_dataset[:1000]
      train_labels_small = train_labels[:1000]
      print "Initialized"
      for step in xrange(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_small[offset:(offset + batch_size), :]
        batch_labels = train_labels_small[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print "Minibatch loss at step", step, ":", l
          print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
          print "Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels)
      print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)
    
setup_graph(0.3)
run_graph()

Answer: The model stops learning when minibatch accuracy is 100%. Without regularization, the validation accuracy gets stuck at 77.3% since the model has basically memorized the training set and applied that to the validation set.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

The extreme overfitting case gets slightly better, but it still ends up with 100% batch accuracy too early.

In [ ]:
def setup_graph(reg_term=0., keep_prob=0.5):
    batch_size = 128
    hidden_size = 1024

    graph = tf.Graph()

    with graph.as_default():
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      weights_h = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_size]))
      biases_h = tf.Variable(tf.zeros([hidden_size]))
      weights_y = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
      biases_y = tf.Variable(tf.truncated_normal([num_labels]))

      # Training computation.
      def single_hidden_layer_logit(input_dataset, keep_prob=keep_prob):
        hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(input_dataset, weights_h) + biases_h), keep_prob)
        logits = tf.matmul(hidden, weights_y) + biases_y
        return logits

      logits = single_hidden_layer_logit(tf_train_dataset, keep_prob=0.5)
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
      loss_with_reg = loss + reg_term * (tf.nn.l2_loss(weights_h) + tf.nn.l2_loss(weights_y))

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss_with_reg)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(single_hidden_layer_logit(tf_valid_dataset, keep_prob=1.0))
      test_prediction = tf.nn.softmax(single_hidden_layer_logit(tf_test_dataset, keep_prob=1.0))

num_steps = 10001
def run_graph():
    with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print "Initialized"
      for step in xrange(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print "Minibatch loss at step", step, ":", l
          print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
          print "Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels)
      print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

for keep_prob in [0.5, 0.75]:
    print "Training with dropout (keep_prob): %s" % keep_prob
    setup_graph(reg_term=0.3, keep_prob=keep_prob)
    run_graph()

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [12]:
num_steps = 64001
batch_size = 128
hidden_size_1 = 1024
hidden_size_2 = 300
hidden_size_3 = 50
reg_term=0.001
keep_prob=0.8

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  weights_h_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size_1]))
  biases_h_1 = tf.Variable(tf.zeros([hidden_size_1]))
  weights_h_2 = tf.Variable(tf.truncated_normal([hidden_size_1, hidden_size_2]))
  biases_h_2 = tf.Variable(tf.zeros([hidden_size_2]))
  weights_h_3 = tf.Variable(tf.truncated_normal([hidden_size_2, hidden_size_3]))
  biases_h_3 = tf.Variable(tf.zeros([hidden_size_3]))
  weights_y = tf.Variable(tf.truncated_normal([hidden_size_3, num_labels]))
  biases_y = tf.Variable(tf.truncated_normal([num_labels]))

  # Training computation.
  def multi_layer_logit(input_dataset, keep_prob=keep_prob):
    hidden_1 = tf.nn.dropout(tf.nn.relu(tf.matmul(input_dataset, weights_h_1) + biases_h_1), keep_prob)
    hidden_2 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden_1, weights_h_2) + biases_h_2), keep_prob)
    hidden_3 = tf.nn.dropout(tf.nn.relu(tf.matmul(hidden_2, weights_h_3) + biases_h_3), keep_prob)      

    logits = tf.matmul(hidden_3, weights_y) + biases_y
    return logits

  logits = multi_layer_logit(tf_train_dataset, keep_prob=0.5)
  epsilon = 1e-9
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits + epsilon, tf_train_labels))
  
  loss_with_reg = loss + reg_term * (
        tf.nn.l2_loss(weights_h_1) +
        tf.nn.l2_loss(weights_h_2) + 
        tf.nn.l2_loss(weights_h_3) +            
        tf.nn.l2_loss(weights_y)
    )

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(0.01, global_step, 10000, 0.95)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_with_reg, global_step=global_step)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(multi_layer_logit(tf_valid_dataset, keep_prob=1.0))
  test_prediction = tf.nn.softmax(multi_layer_logit(tf_test_dataset, keep_prob=1.0))

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)


Initialized
Minibatch loss at step 0 : 34718.1
Minibatch accuracy: 15.6%
Validation accuracy: 9.4%
Minibatch loss at step 1000 : 2.31229
Minibatch accuracy: 10.9%
Validation accuracy: 10.4%
Minibatch loss at step 2000 : 2.3071
Minibatch accuracy: 9.4%
Validation accuracy: 10.4%
Minibatch loss at step 3000 : 2.29897
Minibatch accuracy: 12.5%
Validation accuracy: 10.4%
Minibatch loss at step 4000 : 2.30373
Minibatch accuracy: 7.0%
Validation accuracy: 10.4%
Minibatch loss at step 5000 : 2.30152
Minibatch accuracy: 9.4%
Validation accuracy: 10.4%
Minibatch loss at step 6000 : 2.30267
Minibatch accuracy: 12.5%
Validation accuracy: 10.1%
Minibatch loss at step 7000 : 2.30219
Minibatch accuracy: 15.6%
Validation accuracy: 10.4%
Minibatch loss at step 8000 : 2.3031
Minibatch accuracy: 10.9%
Validation accuracy: 10.1%
Minibatch loss at step 9000 : 2.30252
Minibatch accuracy: 7.8%
Validation accuracy: 10.4%
Minibatch loss at step 10000 : 2.30177
Minibatch accuracy: 10.2%
Validation accuracy: 9.

KeyboardInterrupt: 